# 1. ABC Investments

ABC Inc. is considering several investment options.  Each option has a minimum investment required as well as a maximum investment allowed. These restrictions, along with the expected return are summarized in the following table (figures are in millions of dollars):

|Option | Minimum investment | Maximum investment | Expected return (%)  |
|:-----:|:------------------:|:------------------:|:--------------------:|
|  1    |      3             |      27            |     13               |
|  2    |      2             |      12            |      9               |
|  3    |      9             |      35            |     17               |
|  4    |      5             |      15            |     10               |
|  5    |     12             |      46            |     22               |
|  6    |      4             |      18            |     12               |

Because of the high-risk nature of Option 5, company policy requires that the total amount invested in Option 5 be no more that the combined amount invested in Options 2, 4 and 6.  In addition, if an investment is made in Option 3, it is required that at least a minimum investment be made in Option 6.  ABC has \$80 million to invest and obviously wants to maximize its total expected return on investment.  Which options should ABC invest in, and how much should be invested?

### Model
Input: 
$lows_i$ (The minimum investment amount for each option)
$highs_i$ (The maximum investment amount for each option)
$ret_i$ (The expected return for each option)

Variables: 

$x_i$ (the total amount invested on option i)

$z_i$ (the binary varible, where $z_i=1$ means we invest on this option, $z_i=0$ means not invest on this one)

Model:

$$\begin{aligned}
\text{minimize}\qquad& \sum_i \frac{x_{i}ret_{i}}{100}  \qquad \text{Sum of all the expected returns we could get from the investment} \\
\text{subject to:}\qquad& \sum_i x_{i}   \qquad \text{The total investment amount should less than or equal to 80 million dollors}\\
& x_5 \leq x_2 + x_4 + x_6  \qquad \text{The investment amount on option 5 is no more than the combined amount on options 2, 4 and 6}\\
& z_3 \leq z_6  \qquad \text{if there is an investment in option 3, we must invest in option 6, which means if $z_3 = 1$ then $z_6$ must be equal to 1}\\
& lows_{i}z_{i} \leq x_i \qquad \forall i \text{The minimum investment requirement. If we invest on option i, then $z_i = 1$, so $x_i \geq lows_i$, otherwise $z_i=0$ and $x_i \geq 0$ }\\
& highs_{i}z_{i} \geq x_i \qquad \forall i \text{The maximum investment requirement. If we invest on option i, then $z_i = 1$, so $x_i \leq highs_i$, otherwise $z_i=0$ and $x_i \leq 0$ }\\
& z_i \in \left\{ 0,1\right\} \qquad \forall i\\
\end{aligned}$$

In [1]:
using Pkg
Pkg.add("Cbc")

    Updating registry at `C:\Users\noahx\.julia\registries\General`


    Updating git-repo `https://github.com/JuliaRegistries/General.git`


   Resolving package versions...


   Installed Osi_jll ─────── v0.10800.600+0
   Installed Clp_jll ─────── v100.1700.601+0
   Installed CoinUtils_jll ─ v200.1100.400+0
   Installed Cgl_jll ─────── v0.6000.300+0
   Installed Cbc_jll ─────── v200.1000.501+0
   Installed Cbc ─────────── v1.0.0


    Updating

 `C:\Users\noahx\.julia\environments\v1.6\Project.toml`
  [9961bab8] + Cbc v1.0.0
    Updating `C:\Users\noahx\.julia\environments\v1.6\Manifest.toml`
  [9961bab8] + Cbc v1.0.0
  [38041ee0] + Cbc_jll v200.1000.501+0
  [3830e938] + Cgl_jll v0.6000.300+0
  [06985876] + Clp_jll v100.1700.601+0
  [be027038] + CoinUtils_jll v200.1100.400+0
  [7da25872] + Osi_jll v0.10800.600+0

Precompiling

project...


  ✓ CoinUtils_jll


  ✓ Osi_jll


In [4]:
using JuMP, Cbc
m = Model(Cbc.Optimizer)
set_silent(m)

lows  = [  3,  2,  9,  5, 12,  4 ]  # minimum investment
highs = [ 27, 12, 35, 15, 46, 18 ]  # maximum investment
ret   = [ 13,  9, 17, 10, 22, 12 ]  # expected return

@variable(m, x[1:6] >= 0)             # amount invested in each option
@variable(m, z[1:6], Bin)             # indicator variable: do we invest in option i or not?

@constraint(m, sum(x) <= 80)     # total investment limit

# option 5 is no more than combined amounts from options 2, 4, and 6
@constraint(m, x[5] <= x[2] + x[4] + x[6])

# if there is an investment in option 3, we must invest in option 6.
# in other words: if z[3] = 1, then z[6] = 1.
@constraint(m, z[3] <= z[6])

# variable lower bounds (if z=0 then x=0. if z=1 then low <= x <= hi)
for i = 1:6
    @constraint(m, lows[i]*z[i] <= x[i]  )
    @constraint(m, x[i] <= highs[i]*z[i] )
end

@expression(m, expectedReturn, sum( x[i] * ret[i]/100 for i = 1:6) )

@objective(m, Max, expectedReturn)

optimize!(m)
println("The maximum return on investment (net profit) is: \$", JuMP.objective_value(m), " million.")
println("This is an average return of: ", 100*JuMP.objective_value(m)/JuMP.value(sum(x)), " %")
println("Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of \$).")
JuMP.value.([1:6 z x])

The maximum return on investment (net profit) is: $13.500000000000004 million.
This is an average return of: 16.875000000000007 %
Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of $).


6×3 Matrix{Float64}:
 1.0  0.0   0.0
 2.0  0.0   0.0
 3.0  1.0  35.0
 4.0  1.0   5.0
 5.0  1.0  22.5
 6.0  1.0  17.5